机器学习中特征工程是非常重要的一一个环节，然而实际应用和研究中，特征工程的很多做法却差别很大。

## 标称属性处理

比如原始数据集中有个省份的属性，在训练模型时，如果有能够直接处理字符串特征的模型，就不需要对省份做数值化处理，但如果要用逻辑回归、xgboost等只能学习数值属性的模型时，就需要把省份映射为数值类型的特征。

In [1]:
# coding: utf-8
import pandas as pd
data = {'uid':[1001, 1002, 1003, 1004], 'province': ['广东省', '江西省', '湖北省', '江西省'], 'age': [18,34, 22, 42]}
df = pd.DataFrame(data, columns=['uid', 'province', 'age'])
df

,uid,province,age
0,1001,广东省,18
1,1002,江西省,34
2,1003,湖北省,22
3,1004,江西省,42


在sklearn中有函数可以用LabelEncoder对province进行编码，完成数值化处理。

In [2]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df.province)
df['pro'] = le.transform(df.province)
df

LabelEncoder()

,uid,province,age,pro
0,1001,广东省,18,0
1,1002,江西省,34,1
2,1003,湖北省,22,2
3,1004,江西省,42,1


但是，在实际业务中，模型的训练和线上预测是两个割裂的环节，甚至由不同的同学负责，模型训练阶段的LabelEncoder对线上预测阶段不可见，线上来了一条新数据`[1005,'湖北省',24]`时，并不知道怎么把'湖北省'进行编码。  
所以比较好的办法就是，设定通用的编码方法，比如映射字典，在训练和预测时的数据预处理都用这一套字典：

In [3]:
# 这里用default_dict是为了避免预测数据中出现训练集中没有的省份而导致的keyerror
from collections import defaultdict
pro_dict = {'广东省': 1, 
            '江西省': 2, 
            '湖北省': 3}
pro_dict = defaultdict(int, pro_dict)

在训练阶段，对province按上述字典进行数值化处理：

In [4]:
df['pro'] = df.province.apply(lambda x: pro_dict[x])
df

,uid,province,age,pro
0,1001,广东省,18,1
1,1002,江西省,34,2
2,1003,湖北省,22,3
3,1004,江西省,42,2


预测阶段直接取对应key值得value：

In [5]:
pro_dict['湖北省']

3